Create a model using built-in library from Pytorch.
This code closelt follows nn_tutorial notebook.

In [1]:
import csv, torch, math

### read in target and attr input files
target_file = 'target_in.csv'
attr_file = 'attr_in.csv'

with open(target_file, 'r') as f: 
    reader = csv.reader(f)
    target_input_list = list(reader)
    
with open(attr_file, 'r') as f: 
    reader = csv.reader(f)
    attr_input_list = list(reader)

target_input_list is in list of lists. We want it to be a list of characters.
attr_input is also a list of string lists. We want it to be a list of float lists.

In [2]:
target_input = [''.join(sublist) for sublist in target_input_list]
attr_input = [[float(x) for x in t] for t in attr_input_list]

The current target is a string of the actual sound. We want it to be number of that it is computable when we train the model. We convert as the following:
's': 0
'x': 1
'neither': 2

In [3]:
coded_y = []
for i in target_input:
    if i == 's': 
        coded_y.append(0)
    elif i == 'x':
        coded_y.append(1)
    else:
        coded_y.append(2)


## convert arrays and lists into tensors for training
x_train, y_train = map(torch.tensor, (attr_input, coded_y))

x_train and y_train should be in the right shapes.

In [4]:
print(x_train.shape, y_train.shape)

torch.Size([101, 3545]) torch.Size([101])


Since there are three classes, we set D_out to 3. n is total number of instances and c is the number of attributes in each instance. We use a loss function from torch.nn.functional.

In [33]:
n,c = x_train.shape
D_out = 3
lr = 0.5
epochs = 3
bs = 5

import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

Try a simple linear combination from model() and check if the shapes are as expected.

In [28]:
preds = model(x_train) 
print(x_train.shape, preds.shape, y_train.shape)

torch.Size([101, 3545]) torch.Size([101, 3]) torch.Size([101])


Try running loss function on the inputs

In [29]:
loss_func(preds, y_train)

tensor(32.1340, grad_fn=<NllLossBackward>)

In [30]:
from torch import nn
from torch import optim

class SoundRecognition(nn.Module):
    def __init__(self):
        super().__init__()
        
        ## This one line does the same thing as the two lines below 
        ## and [xb @ self.weights + self.bias]
        self.lin = nn.Linear(c, D_out)
#         self.weights = nn.Parameter(torch.randn(c, D_out) / math.sqrt(c))
#         self.bias = nn.Parameter(torch.zeros(D_out))
        
    def forward(self, xb):
        return self.lin(xb)

## Get the model and optim object that will be used to update model parameters
def get_model():
    model = SoundRecognition()
    return model, optim.SGD(model.parameters(), lr = lr)

In [34]:
model, opt = get_model()

def fit():
    for epoch in range(epochs):
        for i in range((n - 1) // bs + 1):
            start_i = i * bs
            end_i = start_i + bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()
    
    print('Loss: ', loss_func(model(xb), yb))
    print('Train finished')

fit()


Loss:  tensor(0., grad_fn=<NllLossBackward>)
Train finished
